In [1]:
import sqlite3
from copy import deepcopy
# from pxr import Usd, UsdGeom, Gf, Vt, Sdf

In [13]:
def load_table(db_file, table, cn):
   conn = sqlite3.connect(db_file)
   conn.row_factory = sqlite3.Row
   cursor = conn.cursor()
   query = f'SELECT * FROM {table};'
   cursor.execute(query)
   results = cursor.fetchall()
   columns=results[0].keys()
   print(table,len(results), columns)
   res={}
   for r in results:      
      ins={}
      for c in columns[1:]:
         ins[c]=r[c]
      res[r['id']]=type(cn,(),ins)
   cursor.close()
   conn.close()
   return res


database_file = "geometry_atlas.db"

cp_d = load_table(database_file, 'ChildrenPositions','CP')
at_d = load_table(database_file, 'AlignableTransforms','AT')
el_d = load_table(database_file, 'Elements','EL')
fpv_d = load_table(database_file, 'FullPhysVols','FPV')
fun_d = load_table(database_file, 'Functions','FUN')
gnt_d = load_table(database_file, 'GeoNodesTypes','GNT')
it_d = load_table(database_file, 'IdentifierTags','IT')
lv_d = load_table(database_file, 'LogVols','LV')
mat_d = load_table(database_file, 'Materials','MAT')
nt_d = load_table(database_file, 'NameTags','NT')
pv_d = load_table(database_file, 'PhysVols','PV')
rv_d = load_table(database_file, 'RootVolume','RV')
sd_d = load_table(database_file, 'SerialDenominators','SD')
si_d = load_table(database_file, 'SerialIdentifiers','SI')
st_d = load_table(database_file, 'SerialTransformers','ST')
sh_d = load_table(database_file, 'Shapes','SH')
tr_d = load_table(database_file, 'Transforms','TR')


ChildrenPositions 617000 ['id', 'parentId', 'parentTable', 'parentCopyNumber', 'position', 'childTable', 'childId', 'childCopyNumber']
AlignableTransforms 21541 ['id', 'xx', 'xy', 'xz', 'yx', 'yy', 'yz', 'zx', 'zy', 'zz', 'dx', 'dy', 'dz']
Elements 42 ['id', 'name', 'symbol', 'Z', 'A']
FullPhysVols 26144 ['id', 'logvol']
Functions 5214 ['id', 'expression']
GeoNodesTypes 14 ['id', 'nodeType', 'tableName']
IdentifierTags 58934 ['id', 'identifier']
LogVols 58719 ['id', 'name', 'shape', 'material']
Materials 522 ['id', 'name', 'density', 'elements']
NameTags 47417 ['id', 'name']
PhysVols 57675 ['id', 'logvol']
RootVolume 1 ['id', 'volId', 'volTable']
SerialDenominators 1728 ['id', 'baseName']
SerialIdentifiers 5195 ['id', 'baseId']
SerialTransformers 5214 ['id', 'funcId', 'volId', 'volTable', 'copies']
Shapes 119487 ['id', 'type', 'parameters']
Transforms 186717 ['id', 'xx', 'xy', 'xz', 'yx', 'yy', 'yz', 'zx', 'zy', 'zz', 'dx', 'dy', 'dz']


The main table is ChildrensPositions. We go through it and create a tree of objects.

In [17]:
gnt_d[1].nodeType

'GeoPhysVol'

* Only PhysVols and FullPhysVols can have children. 
* Both of them just point to logvol. Multiple PV/FPVs can point to same LV.


In [30]:
def interpretRelation(cp):
    d_parentTable=gnt_d[cp.parentTable].tableName
    d_childTable=gnt_d[cp.childTable].tableName
    print(f'Ptable:{d_parentTable} Pid:{cp.parentId} Pcn:{cp.parentCopyNumber} pos:{cp.position} Ctable:{d_childTable} Cid:{cp.childId}')
    if d_childTable=='NameTags': 
        print(f'Cname:{nt_d[cp.childId].name}')
    if d_childTable=='FullPhysVols': 
        lv=lv_d[fpv_d[cp.childId].logvol]
        print(f'Clogvol:{lv.name}, shape:{sh_d[lv.shape].type}, mat:{lv.material}')
    print()

In [31]:
for cp in cp_d.values():
    if cp.parentId != 1 or cp.parentTable != 1:
        continue
    interpretRelation(cp)

Ptable:PhysVols Pid:1 Pcn:1 pos:1 Ctable:NameTags Cid:1
Cname:Pixel

Ptable:PhysVols Pid:1 Pcn:1 pos:2 Ctable:AlignableTransforms Cid:1

Ptable:PhysVols Pid:1 Pcn:1 pos:3 Ctable:FullPhysVols Cid:1
Clogvol:Pixel, shape:Tube, mat:1

Ptable:PhysVols Pid:1 Pcn:1 pos:4 Ctable:NameTags Cid:4905
Cname:SCT

Ptable:PhysVols Pid:1 Pcn:1 pos:5 Ctable:IdentifierTags Cid:4294

Ptable:PhysVols Pid:1 Pcn:1 pos:6 Ctable:AlignableTransforms Cid:2036

Ptable:PhysVols Pid:1 Pcn:1 pos:7 Ctable:FullPhysVols Cid:4089
Clogvol:SCT_Barrel, shape:Tube, mat:1

Ptable:PhysVols Pid:1 Pcn:1 pos:8 Ctable:NameTags Cid:9358
Cname:SCT

Ptable:PhysVols Pid:1 Pcn:1 pos:9 Ctable:IdentifierTags Cid:15035

Ptable:PhysVols Pid:1 Pcn:1 pos:10 Ctable:AlignableTransforms Cid:8553

Ptable:PhysVols Pid:1 Pcn:1 pos:11 Ctable:FullPhysVols Cid:14654
Clogvol:SCT_ForwardA, shape:Tube, mat:1

Ptable:PhysVols Pid:1 Pcn:1 pos:12 Ctable:NameTags Cid:12354
Cname:SCT

Ptable:PhysVols Pid:1 Pcn:1 pos:13 Ctable:IdentifierTags Cid:18031

Ptabl

In [75]:
class Node:
    def __init__(self):
        self.level=0
        self.type=''
        
    def printLevel(self, end, start=0):
        if self.level>=start and self.level<=end:
            print(self)
        if self.type=='vol':
            for c in self.children:
                c.printLevel(end, start)

class Vol(Node):
    def __init__(self, id, full, lv):
        super().__init__()
        self.type='vol'
        self.id=id
        self.full=full
        self.children=[]
        self.lv=lv
        self.setLV()

    def setLV(self):
        lv=lv_d[self.lv]
        self.lname=lv.name
        self.shape='x'
        self.material='x'
    
    def __str__(self):
        kind=' PV'
        if self.full: kind='FPV'
        return f'{" _ "*self.level}{kind} id:{self.id} name:{self.lname} shape:{self.shape} material:{self.material}, children:{len(self.children)}'
    
    def addChild(self, ch):
        ch.level=self.level+1
        self.children.append(ch)

class AlignableTransform(Node):
    def __init__(self, id):
        super().__init__()
        self.type='at'
        self.id=id
        self.xx=at_d[id].xx
        self.xy=at_d[id].xy
        self.xz=at_d[id].xz
        self.yx=at_d[id].yx
        self.yy=at_d[id].yy
        self.yz=at_d[id].yz
        self.zx=at_d[id].zx
        self.zy=at_d[id].zy
        self.zz=at_d[id].zz
        self.dx=at_d[id].dx
        self.dy=at_d[id].dy
        self.dz=at_d[id].dz
    
    def __str__(self):
        return f'{" _ "*self.level} AT id:{self.id} xx:{self.xx} dx:{self.dx}'

class Transform(Node):
    def __init__(self, id):
        super().__init__()
        self.type='tr'
        self.id=id
        self.xx=tr_d[id].xx
        self.xy=tr_d[id].xy
        self.xz=tr_d[id].xz
        self.yx=tr_d[id].yx
        self.yy=tr_d[id].yy
        self.yz=tr_d[id].yz
        self.zx=tr_d[id].zx
        self.zy=tr_d[id].zy
        self.zz=tr_d[id].zz
        self.dx=tr_d[id].dx
        self.dy=tr_d[id].dy
        self.dz=tr_d[id].dz
    
    def __str__(self):
        return f'{" _ "*self.level} TR id:{self.id} xx:{self.xx} dx:{self.dx}'
        
class NameTag(Node):
    def __init__(self, name):
        super().__init__()
        self.type='nt'
        self.name=name
        
    def __str__(self):
        return f'{" _ "*self.level} NT {self.name}'

class IdentifierTag(Node):
    def __init__(self, identifier):
        super().__init__()
        self.type='it'
        self.identifier=identifier

    def __str__(self):
        return f'{" _ "*self.level} IT {self.identifier}'


In [55]:
def findParent(branch: Vol, pid: int, full: bool) -> Vol:
    if branch.id == pid and branch.full==full:
        return branch
    else:
        for c in branch.children:
            if c.type != 'vol': continue
            findParent(c, pid, full) 

In [76]:
tree=Vol(id=1,full=False,lv=1)

for cp in cp_d.values():
    if cp.parentId != 1 or cp.parentTable != 1:
        continue
    
    p = findParent(tree, cp.parentId, cp.parentTable-1)

    d_childTable=gnt_d[cp.childTable].tableName
    
    if d_childTable=='FullPhysVols':
        p.addChild(Vol(id=cp.childId, full=True, lv=fpv_d[cp.childId].logvol))
    elif d_childTable=='PhysVols':
        p.addChild(Vol(id=cp.childId, full=False, lv=pv_d[cp.childId].logvol))
    elif d_childTable=='AlignableTransforms':
        p.addChild(AlignableTransform(cp.childId))
    elif d_childTable=='Transforms':
        p.addChild(Transform(cp.childId))
    elif d_childTable=='NameTags':
        p.addChild(NameTag(nt_d[cp.childId].name))
    elif d_childTable=='IdentifierTags':
        p.addChild(IdentifierTag(it_d[cp.childId].identifier))
    else:
        print('skipped:', d_childTable)


In [77]:
tree.printLevel(2)

 PV id:1 name:WorldLog shape:x material:x, children:58
 _  NT Pixel
 _  AT id:1 xx:1.0 dx:0.0
 _ FPV id:1 name:Pixel shape:x material:x, children:0
 _  NT SCT
 _  IT 0
 _  AT id:2036 xx:1.0 dx:0.0
 _ FPV id:4089 name:SCT_Barrel shape:x material:x, children:0
 _  NT SCT
 _  IT 2
 _  AT id:8553 xx:1.0 dx:0.0
 _ FPV id:14654 name:SCT_ForwardA shape:x material:x, children:0
 _  NT SCT
 _  IT -2
 _  AT id:11527 xx:-1.0 dx:2.217e-13
 _ FPV id:17628 name:SCT_ForwardC shape:x material:x, children:0
 _  NT TRT
 _  AT id:14501 xx:1.0 dx:0.0
 _ FPV id:20602 name:TRTBarrel shape:x material:x, children:0
 _  NT TRT
 _  AT id:14598 xx:1.0 dx:0.0
 _  IT 0
 _ FPV id:20699 name:TRTEndcapWheelAB shape:x material:x, children:0
 _  NT TRT
 _  AT id:14653 xx:-1.0 dx:0.0
 _  IT 1
 _ FPV id:20874 name:TRTEndcapWheelAB shape:x material:x, children:0
 _  NT InDetServMat
 _  PV id:9058 name:ServLog shape:x material:x, children:0
 _  NT LAr
 _  NT LArBarrel
 _  AT id:14708 xx:1.0 dx:0.0
 _ FPV id:21049 name:LAr:

Need to work on building the tree：
1. Build from ChildrenPosition relationship


Parents are FullPhysVols or PhysVols, should I build a dict for the relationship? What is childCopyNumber?
Combination is done using SerialDenominators?


2. FullPhysVols, PhysVols ->LogVols ->Shapes/Materials


Test Code: Creating Box

In [ ]:
#Test creating box
    path = "assests/shapes/shape_"+ str(Parameters[0])+".usda"
    shape_name = "/shape_"+str(Parameters[0])
    stage = Usd.Stage.CreateNew(path)
    xform = UsdGeom.Xform.Define(stage, shape_name)
    box = UsdGeom.Cube.Define(stage, shape_name+"/geometry")

    key_value_pairs=Parameters[2].split(";")
    box_length=[]
    for value in key_value_pairs:
        box_length.append(float(value.split("=")[1])*2)
    extent = Vt.Vec3fArray(1, Gf.Vec3f(box_length[0],box_length[1],box_length[2]))
    box.GetPrim().CreateAttribute("extent", Sdf.ValueTypeNames.Float3Array).Set(extent)

    # Add the box as a child of the xform
    xform.GetPrim().GetChildren().append(box.GetPrim())

    # Save the USD file
    stage.GetRootLayer().Save()
    
print(Shapes_data[10][2].split(";")[0])
print(Shapes_data[7])
exec(Shapes_data[10][2].split(";")[0])
exec(Shapes_data[10][2].split(";")[1])
print(Shapes_data[opA-1],Shapes_data[opB-1])


In [ ]:
def Create_Single_Shape(Parameters):
    if Parameters[1] == "Tube":
        Create_Tube(Parameters)
    elif Parameters[1] == "Box":
        Create_Box(Parameters)
    elif Parameters[1] == "SimplePolygonBrep":
        Create_SimplePolygonBrep(Parameters)
    elif Parameters[1] == "Trap":
        Create_Trap(Parameters)
    elif Parameters[1] == "Tubs":
        Create_Tubs(Parameters)
    elif Parameters[1] == "Trd":
        Create_Trd(Parameters)
    elif Parameters[1] == "Pgon":
        Create_Pgon(Parameters)
    elif Parameters[1] == "Cons":
        Create_Cons(Parameters)
    elif Parameters[1] == "Para":
        Create_Para(Parameters)
    elif Parameters[1] == "Pcon":
        Create_Pcon(Parameters)
    elif Parameters[1] == "UnidentifiedShape":
        Create_UnidentifiedShape(Parameters)
    else:
        print("Error with creating Single Shape")

In [ ]:
def Create_Tube(Parameters):
    return
def Create_Box(Parameters):
    path = "assests/shapes/shape_"+ str(Parameters[0])+".usda"
    shape_name = "/shape_"+str(Parameters[0])
    
    stage = Usd.Stage.CreateNew(path)
    xform = UsdGeom.Xform.Define(stage, shape_name)
    box = UsdGeom.Cube.Define(stage, shape_name+"/geometry")

    key_value_pairs=Parameters[2].split(";")
    box_length=[]
    for value in key_value_pairs:
        box_length.append(float(value.split("=")[1])*2)
    extent = Vt.Vec3fArray(1, Gf.Vec3f(box_length[0],box_length[1],box_length[2]))
    box.GetPrim().CreateAttribute("extent", Sdf.ValueTypeNames.Float3Array).Set(extent)

    # Add the box as a child of the xform
    xform.GetPrim().GetChildren().append(box.GetPrim())

    # Save the USD file
    stage.GetRootLayer().Save()
    return

def Create_SimplePolygonBrep(Parameters):
    return
def Create_Trap(Parameters):
    return
def Create_Tubs(Parameters):
    return
    
def Create_Trd(Parameters):
    return
    
def Create_Pgon(Parameters):
    return
    
def Create_Cons(Parameters):
    return
    
def Create_Para(Parameters):
    return
    
def Create_Pcon(Parameters):
    return
    
def Create_UnidentifiedShape(Parameters):
    return


In [ ]:
def Combine_Two_Shapes(Parameters,Shapes_data):
    #print(Parameters[1]+ " Two Shapes " +Parameters[2].split(";")[0] + " "+Parameters[2].split(";")[1])
    if Parameters[1] == "Shift":
        Shift_Shape(Parameters,Shapes_data)
    elif Parameters[1] =="Union":
        Union_Shape(Parameters,Shapes_data)
    elif Parameters[1] =="Subtraction":
        Subtraction_Shape(Parameters,Shapes_data)
    elif Parameters[1] =="Intersection":
        Intersection_Shape(Parameters,Shapes_data)
    else:
        print("Error with Combine two shapes")

In [ ]:
def Shift_Shape(Parameters,Shapes_data):
    return
def Union_Shape(Parameters,Shapes_data):
    return
def Subtraction_Shape(Parameters,Shapes_data):
    return
def Intersection_Shape(Parameters,Shapes_data):
    return

In [ ]:
for row in data:
    if row[1] in {"Tube","Box","SimplePolygonBrep","Trap","Tubs","Trd","Pgon","Cons","Para","Pcon","UnidentifiedShape"}:
        Create_Single_Shape(row)
    elif row[1] in {"Shift","Union","Subtraction","Intersection"}:
        Combine_Two_Shapes(row,Shapes_data)        
    else:
        print("Error with creating shape")